# Merging all the Data from the Groups with and without Randomization

Completely done in Python. There ore no steps required in Excel

v1_03.04.2024

In [1]:
import pandas as pd

In [2]:
# Load the datasets
df1 = pd.read_csv('data_merged_1st.csv')
df2 = pd.read_csv('data_merged_2nd.csv')

In [3]:
# Identify all unique columns across both datasets
all_columns = sorted(set(df1.columns).union(set(df2.columns)))

In [4]:
# Add missing columns to each dataframe with filler values where necessary
for col in all_columns:
    if col not in df1.columns:
        df1[col] = 0  
    if col not in df2.columns:
        df2[col] = 0

In [5]:
# Reorder columns to match across both dataframes, using the sorted list of all unique columns
df1 = df1.reindex(columns=all_columns)
df2 = df2.reindex(columns=all_columns)

In [6]:
# Concatenate the datasets
merged_df = pd.concat([df1, df2], ignore_index=True)

In [7]:
# Fill any other missing values that might still exist after alignment with 0 (MAYBE TEST FIRST WITH SOME CLEAR VALUE)
merged_df.fillna(0, inplace=True)

In [8]:
# Look at the merged dataset
merged_df.head()

,AGE,AILiteracy[Det],AILiteracy[Eth],AILiteracy[Kno],AILiteracy[Use],All_Prompts,Average_Word_Count,CMV,Edu_Abitur_oder_Fachabitur,Edu_Bachelor,...,WORK,group_nr,syst_total_time,time_demogr,time_pers,time_survey1,time_survey2,time_task1,time_task2,total_time
0,22,4.000000,2.5,3.333333,3.166667,Ich benötige Unterstützung bei der Entwicklung...,39.333333,4.666667,0,1,...,25,1,35.038667,9.134167,1.056333,1.290833,11.509667,11.980833,0.066833,35.0
1,20,3.333333,3.5,4.833333,5.000000,\nHilf mir diese Aufgabe zu lösen:\n\n\nEs ist...,108.750000,2.000000,1,0,...,15,3,23.169333,2.530833,1.257667,1.443167,11.385000,6.372500,0.180167,23.0
2,20,3.333333,2.0,4.333333,3.833333,Wie wichtig ist Nachhaltigkeit heute || wie ka...,11.571429,3.666667,1,0,...,20,3,34.525167,3.595667,1.301500,1.658333,2.625667,10.433000,14.911000,34.0
3,21,4.666667,3.0,4.500000,4.000000,"Hey ChatGPT, du bist ein Marketing-Genie, das ...",55.750000,4.000000,1,0,...,30,4,46.279833,1.717167,0.610833,0.892167,1.821167,21.664167,19.574333,46.0
4,25,4.000000,3.0,3.833333,4.166667,"Hallo, kannst du mir ein Kommunikationskonzept...",20.500000,4.333333,0,0,...,60,3,28.102000,3.470833,1.178833,1.634500,2.959667,12.412667,6.445500,28.0


In [9]:
# Adress Radomization by exchaning values from the following columns if the 'group_nr' values are 5, 6, 7, and 8 (from 1 to 4 the order is already correct)
# Define the columns to swap in pairs
swap_pairs = [
    ('JC1[HRJD]', 'JC2[HRJD]'),
    ('JC1[IStR]', 'JC2[IStR]'),
    ('MANI1', 'MANI2'),
    ('PROD1', 'PROD2'),
    ('SDT1[AUT]', 'SDT2[AUT]'),
    ('SDT1[COM]', 'SDT2[COM]'),
    ('SE1', 'SE2'),
    ('TASKDIF1', 'TASKDIF2'),
    ('time_survey1', 'time_survey2'),
    ('time_task1', 'time_task2')
]

# Loop through each pair and swap their values for rows with group_nr in 5, 6, 7, 8
for col1, col2 in swap_pairs:
    # Identify rows where swapping is needed
    mask = merged_df['group_nr'].isin([5, 6, 7, 8])
    
    # Perform the swap
    merged_df.loc[mask, [col1, col2]] = merged_df.loc[mask, [col2, col1]].values

In [10]:
# Create a binary 'Prompt_Support' column with 0 as default value
merged_df['Prompt_Support'] = 0

# Update 'Prompt_Support' to 1 where 'group_nr' is 2, 4, 6, 8 (= Prompting Framework used)
merged_df.loc[merged_df['group_nr'].isin([2, 4, 6, 8]), 'Prompt_Support'] = 1

In [11]:
# Rename some columns for convinience
merged_df.rename(columns={
    'Average_Word_Count': 'Average_Word_Count_Prompts',
    'Total_Word_Count': 'Total_Word_Count_Prompt',
    'VPNCode': 'ID',
    'group_nr': 'Group_Nr'
}, inplace=True)

In [12]:
# Change order of columns to a more desired one in order to get a better overview
new_column_order = [
    'ID', 'AGE', 'WORK', 
    'Gender_Männlich', 'Gender_Weiblich', 'Edu_Abitur_oder_Fachabitur', 'Edu_Bachelor', 'Edu_Master',
    'AILiteracy[Use]', 'AILiteracy[Kno]', 'AILiteracy[Det]', 'AILiteracy[Eth]',
    'PGAT', 'NGAT', 'CMV', 'NEO[E]', 'NEO[A]', 'NEO[C]', 'NEO[N]', 'NEO[O]', 'Group_Nr', 'Prompt_Support',
    'JC1[IStR]', 'JC1[HRJD]', 'SE1', 'SDT1[AUT]', 'SDT1[COM]', 'PROD1', 'TASKDIF1', 'MANI1',
    'JC2[IStR]', 'JC2[HRJD]', 'SE2', 'SDT2[AUT]', 'SDT2[COM]', 'PROD2', 'TASKDIF2', 'MANI2', 'All_Prompts', 'Prompt_Count', 'Average_Word_Count_Prompts', 'First_Prompt_Word_Count', 'Total_Word_Count_Prompt',
    'total_time', 'syst_total_time', 'time_demogr', 'time_pers', 'time_task1', 'time_survey1', 'time_task2', 'time_survey2'
]

merged_df = merged_df[new_column_order]

In [13]:
# Maybe not the big focus for my main analysis but could be useful for the explorative analysis: Time columns don't seem always correct, so some changes will be implemented

# Drop the 'total_time' column as it exists twice (= syst_total_time)
merged_df.drop(columns=['total_time'], inplace=True)

# Rename 'syst_total_time' column to 'time_total' for coherency
merged_df.rename(columns={'syst_total_time': 'time_total'}, inplace=True)

# Compare 'time_task1' with 'time_survey1' and swap if 'time_task1' is smaller 
# Reason: I could be wrong but looking at the values from the corrresponding columns, it does seem more resonable that someone required more time to complete the task than the survey (further, sometimes absurd values like 4 seconds for the task but 11 minutes for the survey). This is not the case for every row
mask = merged_df['time_task1'] < merged_df['time_survey1']
merged_df.loc[mask, ['time_task1', 'time_survey1']] = merged_df.loc[mask, ['time_survey1', 'time_task1']].values

# Compare 'time_task2' with 'time_survey2' and swap if 'time_task2' is smaller 
# Same reason as above
mask = merged_df['time_task2'] < merged_df['time_survey2']
merged_df.loc[mask, ['time_task2', 'time_survey2']] = merged_df.loc[mask, ['time_survey2', 'time_task2']].values

In [14]:
# Look at the new dataset
merged_df.head()

,ID,AGE,WORK,Gender_Männlich,Gender_Weiblich,Edu_Abitur_oder_Fachabitur,Edu_Bachelor,Edu_Master,AILiteracy[Use],AILiteracy[Kno],...,Average_Word_Count_Prompts,First_Prompt_Word_Count,Total_Word_Count_Prompt,time_total,time_demogr,time_pers,time_task1,time_survey1,time_task2,time_survey2
0,ABO2606,22,25,1,0,0,1,0,3.166667,3.333333,...,39.333333,100,118,35.038667,9.134167,1.056333,11.980833,1.290833,11.509667,0.066833
1,APE2704,20,15,1,0,1,0,0,5.000000,4.833333,...,108.750000,128,435,23.169333,2.530833,1.257667,6.372500,1.443167,11.385000,0.180167
2,SSG0102,20,20,1,0,1,0,0,3.833333,4.333333,...,11.571429,5,81,34.525167,3.595667,1.301500,10.433000,1.658333,14.911000,2.625667
3,ECH2807,21,30,1,0,1,0,0,4.000000,4.500000,...,55.750000,57,223,46.279833,1.717167,0.610833,21.664167,0.892167,19.574333,1.821167
4,AST1210,25,60,1,0,0,0,0,4.166667,3.833333,...,20.500000,27,41,28.102000,3.470833,1.178833,12.412667,1.634500,6.445500,2.959667


In [15]:
# Renaming columns to remove brackets -> For R
# merged_df.columns = [col.replace('[', '').replace(']', '') for col in merged_df.columns]

In [16]:
# Save DF in a new CSV-File 
merged_df.to_csv('data_merged_all_R.csv', index=False, encoding='utf-8-sig', sep=',')